# Johansen Cointegration

In [36]:
import binance_api
import numpy as np
import pandas as pd
import math

import statsmodels
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as stat
import statsmodels.tsa.stattools as ts

import matplotlib.pyplot as plt
from datetime import datetime, date

import seaborn as sb
# https://www.marketcalls.in/amibroker/computing-cointegration-and-augmented-dickey-fuller-test-in-amibroker-using-python.html#:~:text=Augmented%20Dicky%20Fuller%20test%20is,stationary%20and%20cointegrated%20or%20not.&text=The%20Augmented%20Dicky%20Fuller%20test,want%20to%20reject%20this%20hypothesis.


In [156]:
# default values
nbr_of_coins = 50
interval = '1h'
lookback = 14 * 24

#startTime = round(datetime(2021, 12, 5).timestamp()) * 1000
endTime = round(datetime(2022, 2, 16).timestamp()) * 1000

In [160]:
# Get all coins, sort them by volume and keep the top x nbr of coins
tickers = pd.DataFrame(binance_api.get_all_tickers())
tickers['volume_usd'] = tickers.apply(lambda row: float(row['volume']) * float(row['lastPrice']), axis=1)
tickers = tickers.sort_values(by='volume_usd', ascending=False)
top_tickers = tickers[:nbr_of_coins]

top_tickers = top_tickers[top_tickers['symbol'] != 'PEOPLEUSDT']
top_tickers = top_tickers[top_tickers['symbol'] != 'BTCUSDT_220325']
top_tickers = top_tickers[top_tickers['symbol'] != 'ROSEUSDT']
top_tickers = top_tickers[top_tickers['symbol'] != 'API3USDT']
print(top_tickers['symbol'].values.tolist())

['BTCUSDT', 'ETHUSDT', 'LUNAUSDT', 'BTCBUSD', 'SOLUSDT', 'GALAUSDT', '1000SHIBUSDT', 'AVAXUSDT', 'ADAUSDT', 'XRPUSDT', 'FTMUSDT', 'ETHBUSD', 'SANDUSDT', 'MANAUSDT', 'BNBUSDT', 'MATICUSDT', 'DOTUSDT', 'ATOMUSDT', 'LINKUSDT', 'DOGEUSDT', 'NEARUSDT', 'ALICEUSDT', 'LTCUSDT', 'AXSUSDT', 'ETCUSDT', 'CRVUSDT', 'BCHUSDT', 'EOSUSDT', 'ONEUSDT', 'THETAUSDT', 'EGLDUSDT', 'WAVESUSDT', 'FILUSDT', 'XTZUSDT', 'DYDXUSDT', 'ICPUSDT', 'LRCUSDT', 'SUSHIUSDT', 'TRXUSDT', 'SOLBUSD', 'UNIUSDT', 'VETUSDT', 'ENJUSDT', 'AAVEUSDT', 'BNBBUSD', 'NEOUSDT']


In [161]:
historical_data = {}
for i in range(0, len(top_tickers)):
    historical_data[top_tickers.iloc[i]['symbol']] = pd.DataFrame(binance_api.get_historical_prices(top_tickers.iloc[i]['symbol'], interval, None, endTime))[4]

historical_prices = pd.DataFrame(historical_data)
historical_prices

,BTCUSDT,ETHUSDT,LUNAUSDT,BTCBUSD,SOLUSDT,GALAUSDT,1000SHIBUSDT,AVAXUSDT,ADAUSDT,XRPUSDT,...,LRCUSDT,SUSHIUSDT,TRXUSDT,SOLBUSD,UNIUSDT,VETUSDT,ENJUSDT,AAVEUSDT,BNBBUSD,NEOUSDT
0,46682.79,3820.34,85.2710,46687.3,168.3000,0.42913,0.032670,105.2220,1.33080,0.8288,...,2.05519,8.5940,0.07712,168.2600,19.7940,0.093850,2.72510,255.610,512.710,27.088
1,46611.46,3819.25,85.0630,46608.3,168.3800,0.42901,0.032685,105.2480,1.32750,0.8279,...,2.03894,8.6750,0.07703,168.3600,19.6680,0.094220,2.71680,258.850,512.630,27.098
2,46308.23,3792.03,83.5770,46296.5,167.7600,0.42523,0.032538,104.3480,1.32010,0.8263,...,2.02944,8.8870,0.07674,167.7100,19.2950,0.095440,2.69630,256.990,511.300,26.944
3,46007.99,3771.55,81.9240,46006.4,163.5100,0.41381,0.031483,102.2710,1.30680,0.8206,...,1.97647,8.4790,0.07616,163.5000,19.0340,0.093460,2.62870,252.660,507.170,26.578
4,45922.29,3746.78,82.5760,45916.6,163.9000,0.41701,0.031780,102.1060,1.30930,0.8215,...,1.98644,8.6420,0.07565,163.9000,19.1810,0.094440,2.64100,251.620,507.860,26.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,43987.20,3138.17,56.1240,44012.6,103.1200,0.34460,0.030578,93.9700,1.09750,0.8377,...,0.96750,4.3670,0.06629,103.1900,11.2680,0.060010,1.95930,169.590,432.420,22.457
996,44066.10,3154.30,56.1880,44085.6,103.2900,0.34621,0.030776,94.2400,1.09670,0.8362,...,0.98160,4.3560,0.06616,103.3600,11.2650,0.060200,1.96980,170.570,431.940,23.895
997,43970.90,3140.61,55.9860,43997.8,102.7500,0.34247,0.030671,93.8700,1.09680,0.8353,...,0.97500,4.3450,0.06618,102.8200,11.1860,0.059900,1.97230,171.510,430.870,23.395
998,43886.90,3123.81,56.1570,43920.8,101.9100,0.34211,0.030319,94.3400,1.08740,0.8324,...,0.96900,4.3070,0.06585,101.9400,11.1890,0.059730,1.95720,169.680,430.060,24.109


In [162]:
# Example Johansen
df = pd.DataFrame({'x': pd.to_numeric(historical_prices['BTCUSDT']), 'y': pd.to_numeric(historical_prices['ETHUSDT'])})
jres = coint_johansen(df, 0, 1)
print('trace: {}, critial value at 0.05%: {}, reject null? {}'.format(jres.trace_stat[0], jres.cvt[0][1], jres.trace_stat[0] > jres.cvt[0][1]))


# From my comprehension, we reject the null hypothesis if the trace of max-eigen statistic is higher than the value at the 0.05% critical value

trace: 14.975155283988379, critial value at 0.05%: 15.4943, reject null? False


In [163]:
# Cointegration Matrix
symbols = historical_prices.columns.tolist()

# short
coint_matrix = pd.DataFrame(index=symbols, columns=symbols)
for i in range(0, len(coint_matrix)):
    for j in range(0, len(coint_matrix)):
        if i != j:
            #print('{},{}'.format(i,j))
            data_1 = np.asarray(historical_prices[coint_matrix.index[i]].tail(lookback)).astype(float)
            data_2 = np.asarray(historical_prices[coint_matrix.index[j]].tail(lookback)).astype(float)
            if len(data_1) == len(data_2):
                df = pd.DataFrame({'x': pd.to_numeric(historical_prices['BTCUSDT']), 'y': pd.to_numeric(historical_prices['ETHUSDT'])})
                jres = coint_johansen(df, 0, 1)
                # From my comprehension, we reject the null hypothesis if the trace of max-eigen statistic is higher than the value at the 0.05% critical value
                coint_matrix[coint_matrix.index[i]][coint_matrix.index[j]] = jres.trace_stat[0] > jres.cvt[0][1]

In [164]:
coint_matrix

,BTCUSDT,ETHUSDT,LUNAUSDT,BTCBUSD,SOLUSDT,GALAUSDT,1000SHIBUSDT,AVAXUSDT,ADAUSDT,XRPUSDT,...,LRCUSDT,SUSHIUSDT,TRXUSDT,SOLBUSD,UNIUSDT,VETUSDT,ENJUSDT,AAVEUSDT,BNBBUSD,NEOUSDT
BTCUSDT,NaN,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ETHUSDT,False,NaN,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
LUNAUSDT,False,False,NaN,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
BTCBUSD,False,False,False,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SOLUSDT,False,False,False,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
GALAUSDT,False,False,False,False,False,NaN,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000SHIBUSDT,False,False,False,False,False,False,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AVAXUSDT,False,False,False,False,False,False,False,NaN,False,False,...,False,False,False,False,False,False,False,False,False,False
ADAUSDT,False,False,False,False,False,False,False,False,NaN,False,...,False,False,False,False,False,False,False,False,False,False
XRPUSDT,False,False,False,False,False,False,False,False,False,NaN,...,False,False,False,False,False,False,False,False,False,False
